<a href="https://colab.research.google.com/github/feliciahf/NLP-Project/blob/main/amazon/colab_FH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes

Check whether stemming & lemmatization make difference

Do we need bigram/trigram frequencies (within titles)?

Do we need dataframe including cleaned data & genres? (which cleaned data?) -> create dictionary of lists maybe
https://www.geeksforgeeks.org/python-ways-to-create-a-dictionary-of-lists/

How do we incorporate POS-tagging & NER in classifier?

# Importing & Cleaning data

This section imports the data into a pandas dataframe and goes through the following preprocessing steps:

-Case collapsing 
-Remove punctuation 
-Tokenization 
-N-Grams: bigrams and trigrams 
-Stemming -> check whether this makes a difference 
-Lemmatization -> check whether this makes a difference 
-Part-of-speech (POS) tagging 
-Named entity recognition (NER) 

## Import Data
Imports data as pandas dataframe
Create list of title and genre columns from original data
Create list including all 32 genres

In [1]:
from google.colab import files
uploaded = files.upload()

Saving book32listing.csv to book32listing.csv


In [2]:
import pandas as pd
df = pd.read_csv("book32listing.csv", encoding='latin1', header=None)
df1 = df[[3,6]] # only columns with titles and genres
df1.columns = ['title', 'genre']
print(df1)

                                                    title      genre
0                         Mom's Family Wall Calendar 2016  Calendars
1                         Doug the Pug 2016 Wall Calendar  Calendars
2       Moleskine 2016 Weekly Notebook, 12M, Large, Bl...  Calendars
3                 365 Cats Color Page-A-Day Calendar 2016  Calendars
4                    Sierra Club Engagement Calendar 2016  Calendars
...                                                   ...        ...
207567  ADC the Map People Washington D.C.: Street Map...     Travel
207568  Washington, D.C., Then and Now: 69 Sites Photo...     Travel
207569  The Unofficial Guide to Washington, D.C. (Unof...     Travel
207570      Washington, D.C. For Dummies (Dummies Travel)     Travel
207571  Fodor's Where to Weekend Around Boston, 1st Ed...     Travel

[207572 rows x 2 columns]


In [3]:
titles = df1['title'] # list of all titles
titles1 = titles.values.tolist() # change to list of strings
print(titles1[0:6]) # test whether it worked

["Mom's Family Wall Calendar 2016", 'Doug the Pug 2016 Wall Calendar', 'Moleskine 2016 Weekly Notebook, 12M, Large, Black, Soft Cover (5 x 8.25)', '365 Cats Color Page-A-Day Calendar 2016', 'Sierra Club Engagement Calendar 2016', 'Sierra Club Wilderness Calendar 2016']


In [4]:
genres = df1['genre']
genres = genres.values.tolist()
genres = pd.DataFrame(genres)

In [5]:
df1.genre.unique() # list of all possible genres

array(['Calendars', 'Comics & Graphic Novels', 'Test Preparation',
       'Mystery, Thriller & Suspense', 'Science Fiction & Fantasy',
       'Romance', 'Humor & Entertainment', 'Literature & Fiction',
       'Gay & Lesbian', 'Engineering & Transportation',
       'Cookbooks, Food & Wine', 'Crafts, Hobbies & Home',
       'Arts & Photography', 'Education & Teaching',
       'Parenting & Relationships', 'Self-Help', 'Computers & Technology',
       'Medical Books', 'Science & Math', 'Health, Fitness & Dieting',
       'Business & Money', 'Law', 'Biographies & Memoirs', 'History',
       'Politics & Social Sciences', 'Reference',
       'Christian Books & Bibles', 'Religion & Spirituality',
       'Sports & Outdoors', 'Teen & Young Adult', "Children's Books",
       'Travel'], dtype=object)

## Case Collapsing
Change all uppercase to lowercase letters

In [6]:
case_collap = map(lambda x:x.lower(), titles1)
case_collap_list = list(case_collap)
print(case_collap_list[0:6])

["mom's family wall calendar 2016", 'doug the pug 2016 wall calendar', 'moleskine 2016 weekly notebook, 12m, large, black, soft cover (5 x 8.25)', '365 cats color page-a-day calendar 2016', 'sierra club engagement calendar 2016', 'sierra club wilderness calendar 2016']


## Remove Punctuation
Remove punctuation by creating translation table
Punctuation to be removed is given in string: string.punctuation

In [7]:
import string
trans = str.maketrans('', '', string.punctuation)
rem_punct = [s.translate(trans) for s in case_collap_list]
print(rem_punct[0:6])

['moms family wall calendar 2016', 'doug the pug 2016 wall calendar', 'moleskine 2016 weekly notebook 12m large black soft cover 5 x 825', '365 cats color pageaday calendar 2016', 'sierra club engagement calendar 2016', 'sierra club wilderness calendar 2016']


## Tokenization
Split all titles into words
Output: list of lists of strings

In [8]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
tokenized_titles = [word_tokenize(i) for i in rem_punct]
print(tokenized_titles[:10])

[['moms', 'family', 'wall', 'calendar', '2016'], ['doug', 'the', 'pug', '2016', 'wall', 'calendar'], ['moleskine', '2016', 'weekly', 'notebook', '12m', 'large', 'black', 'soft', 'cover', '5', 'x', '825'], ['365', 'cats', 'color', 'pageaday', 'calendar', '2016'], ['sierra', 'club', 'engagement', 'calendar', '2016'], ['sierra', 'club', 'wilderness', 'calendar', '2016'], ['thomas', 'kinkade', 'the', 'disney', 'dreams', 'collection', '2016', 'wall', 'calendar'], ['ansel', 'adams', '2016', 'wall', 'calendar'], ['dilbert', '2016', 'daytoday', 'calendar'], ['mary', 'engelbreit', '2016', 'deluxe', 'wall', 'calendar', 'never', 'give', 'up']]


## N-Grams: Bigrams and Trigrams
Create bigrams + trigrams
(could be done in one go e.g. n=2 for bigrams, n=3 for trigrams etc.

In [10]:
# bigrams
token_bigram = []
for title in tokenized_titles:
    title_bigram = []
    for w in range(len(title) - 1):
        title_bigram.append([title[w], title[w + 1]])
    token_bigram.append(title_bigram)
print(token_bigram[:6]) # test whether working

[[['moms', 'family'], ['family', 'wall'], ['wall', 'calendar'], ['calendar', '2016']], [['doug', 'the'], ['the', 'pug'], ['pug', '2016'], ['2016', 'wall'], ['wall', 'calendar']], [['moleskine', '2016'], ['2016', 'weekly'], ['weekly', 'notebook'], ['notebook', '12m'], ['12m', 'large'], ['large', 'black'], ['black', 'soft'], ['soft', 'cover'], ['cover', '5'], ['5', 'x'], ['x', '825']], [['365', 'cats'], ['cats', 'color'], ['color', 'pageaday'], ['pageaday', 'calendar'], ['calendar', '2016']], [['sierra', 'club'], ['club', 'engagement'], ['engagement', 'calendar'], ['calendar', '2016']], [['sierra', 'club'], ['club', 'wilderness'], ['wilderness', 'calendar'], ['calendar', '2016']]]


In [11]:
# trigrams
token_trigram = []
for title in tokenized_titles:
    title_trigram = []
    for w in range(len(title) - 2):
        title_trigram.append([title[w], title[w + 1], title[w + 2]])
    token_trigram.append(title_trigram)
print(token_trigram[:6]) # test whether working

[[['moms', 'family', 'wall'], ['family', 'wall', 'calendar'], ['wall', 'calendar', '2016']], [['doug', 'the', 'pug'], ['the', 'pug', '2016'], ['pug', '2016', 'wall'], ['2016', 'wall', 'calendar']], [['moleskine', '2016', 'weekly'], ['2016', 'weekly', 'notebook'], ['weekly', 'notebook', '12m'], ['notebook', '12m', 'large'], ['12m', 'large', 'black'], ['large', 'black', 'soft'], ['black', 'soft', 'cover'], ['soft', 'cover', '5'], ['cover', '5', 'x'], ['5', 'x', '825']], [['365', 'cats', 'color'], ['cats', 'color', 'pageaday'], ['color', 'pageaday', 'calendar'], ['pageaday', 'calendar', '2016']], [['sierra', 'club', 'engagement'], ['club', 'engagement', 'calendar'], ['engagement', 'calendar', '2016']], [['sierra', 'club', 'wilderness'], ['club', 'wilderness', 'calendar'], ['wilderness', 'calendar', '2016']]]


## Stemming
Test this out to see whether it makes a difference in final classifier

PorterStemmer (one algorithm for stemming; less aggressive than LancasterStemming)

Create empty list to contain lists of stems in each title
Create empty list for stems of title
Add each stemmed word in title to second list
Append list of stemmed words to first list

In [12]:
from nltk.stem import PorterStemmer

porter = PorterStemmer()
stems = []   
for title in tokenized_titles:
    stems_title = []
    for word in title:
        stems_title.append(porter.stem(word))
    stems.append(stems_title)
    
print(stems[0:6]) # test whether it works

[['mom', 'famili', 'wall', 'calendar', '2016'], ['doug', 'the', 'pug', '2016', 'wall', 'calendar'], ['moleskin', '2016', 'weekli', 'notebook', '12m', 'larg', 'black', 'soft', 'cover', '5', 'x', '825'], ['365', 'cat', 'color', 'pageaday', 'calendar', '2016'], ['sierra', 'club', 'engag', 'calendar', '2016'], ['sierra', 'club', 'wilder', 'calendar', '2016']]


## Lemmatization
Same principle as with stemming
Test this out to see whether it makes a difference in final classifier

In [13]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
lemmas = []
for title in tokenized_titles:
    lemmas_title = []
    for word in title:
        lemmas_title.append(lemmatizer.lemmatize(word))
    lemmas.append(lemmas_title)
    
print(lemmas[0:6]) # test whether it works

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[['mom', 'family', 'wall', 'calendar', '2016'], ['doug', 'the', 'pug', '2016', 'wall', 'calendar'], ['moleskine', '2016', 'weekly', 'notebook', '12m', 'large', 'black', 'soft', 'cover', '5', 'x', '825'], ['365', 'cat', 'color', 'pageaday', 'calendar', '2016'], ['sierra', 'club', 'engagement', 'calendar', '2016'], ['sierra', 'club', 'wilderness', 'calendar', '2016']]


## Part-of-Speech (POS) Tagging
Create list of lists with tokens and their corresponding part-of-speech tag in each title

In [14]:
nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag

postag = []
for title in tokenized_titles:
    postag.append(nltk.pos_tag(title))
    
print(postag[0:6]) # testing whether postag worked

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[[('moms', 'NNS'), ('family', 'NN'), ('wall', 'NN'), ('calendar', 'NN'), ('2016', 'CD')], [('doug', 'VB'), ('the', 'DT'), ('pug', 'NN'), ('2016', 'CD'), ('wall', 'NN'), ('calendar', 'NN')], [('moleskine', 'NN'), ('2016', 'CD'), ('weekly', 'JJ'), ('notebook', 'NN'), ('12m', 'CD'), ('large', 'JJ'), ('black', 'JJ'), ('soft', 'JJ'), ('cover', 'NN'), ('5', 'CD'), ('x', 'JJ'), ('825', 'CD')], [('365', 'CD'), ('cats', 'NNS'), ('color', 'VBP'), ('pageaday', 'IN'), ('calendar', 'NN'), ('2016', 'CD')], [('sierra', 'NN'), ('club', 'NN'), ('engagement', 'NN'), ('calendar', 'NN'), ('2016', 'CD')], [('sierra', 'NN'), ('club', 'NN'), ('wilderness', 'NN'), ('calendar', 'NN'), ('2016', 'CD')]]


## Named Entity Recognition (NER)
Create list of lists with words, correspondent POS and named entity tags for each title
This uses postags created in previous step

In [15]:
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk import ne_chunk
from nltk.chunk import tree2conlltags

ner = []
for title in titles1:
    ner.append(tree2conlltags(ne_chunk(pos_tag(word_tokenize(title)))))
    
print(ner[0:6]) # test whether NER worked

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[[('Mom', 'NNP', 'B-PERSON'), ("'s", 'POS', 'O'), ('Family', 'NNP', 'B-PERSON'), ('Wall', 'NNP', 'I-PERSON'), ('Calendar', 'NNP', 'I-PERSON'), ('2016', 'CD', 'O')], [('Doug', 'NNP', 'O'), ('the', 'DT', 'O'), ('Pug', 'NNP', 'O'), ('2016', 'CD', 'O'), ('Wall', 'NNP', 'B-FACILITY'), ('Calendar', 'NNP', 'I-FACILITY')], [('Moleskine', 'NN', 'O'), ('2016', 'CD', 'O'), ('Weekly', 'NNP', 'O'), ('Notebook', 'NNP', 'O'), (',', ',', 'O'), ('12M', 'CD', 'O'), (',', ',', 'O'), ('Large', 'NNP', 'B-PERSON'), (',', ',', 'O'), ('Black', 'NNP', 'B-PERSON'), (',', ',', 'O'), ('Soft', 'NNP', 'B-PERSON'), ('Cover', 'NNP', 'I-PERSON'), ('(', '(', 'O'), ('5', 'CD', 'O'), ('x', 'RB', 'O'), ('8.25', 'CD', 'O'), (')', ')', 'O')], [('365', 'CD', 'O'), ('Cats', 'NNPS', '

# Classifiers
Check whether stemming/lemmatization make difference in final classifiers
-> does it improve/worsen classifier?


-Naive Bayes (bag-of-words)

-BERT (language model)

https://towardsdatascience.com/text-classification-with-nlp-tf-idf-vs-word2vec-vs-bert-41ff868d1794

In [16]:
# create dataframe containing tokenized titles and genres
tok_title = pd.DataFrame({0: tokenized_titles})
data_in = [tok_title[0], df1["genre"]]
headers = ["titles", "genres"]

data = pd.concat(data_in, axis=1, keys=headers)
print(data[:5])

                                              titles     genres
0               [moms, family, wall, calendar, 2016]  Calendars
1             [doug, the, pug, 2016, wall, calendar]  Calendars
2  [moleskine, 2016, weekly, notebook, 12m, large...  Calendars
3       [365, cats, color, pageaday, calendar, 2016]  Calendars
4         [sierra, club, engagement, calendar, 2016]  Calendars


In [17]:
# split data into train and test
import numpy as np

test_pct=0.2 # split into 80/20%

# create mask
mask = np.random.choice([0, 1], p=[1 - test_pct, test_pct], size=data.shape[0])

# apply mask
data["mask"] = mask
test = data[data["mask"] == 1]
train = data[data["mask"] == 0]

# removing column
test = test.drop("mask", axis="columns").reset_index()
train = train.drop("mask", axis="columns").reset_index()

# remove original indexing data (otherwise we have double indexing)
test = test.drop("index", axis="columns")
train = train.drop("index", axis="columns")

## Naive Bayes classifier

split into training and test data -> 80% / 20%
use n-grams ?

build vocabulary -> need token frequencies across all titles (create sublist with tokenized_titles)
vocabulary for each genre

In [18]:
from collections import Counter

In [19]:
# token frequencies within each title
unigram_count = []
for title in tokenized_titles:
    uni_title = Counter()
    for i in title:
        uni_title[i] += 1
    unigram_count.append(uni_title)

print(unigram_count[:5]) # test

[Counter({'moms': 1, 'family': 1, 'wall': 1, 'calendar': 1, '2016': 1}), Counter({'doug': 1, 'the': 1, 'pug': 1, '2016': 1, 'wall': 1, 'calendar': 1}), Counter({'moleskine': 1, '2016': 1, 'weekly': 1, 'notebook': 1, '12m': 1, 'large': 1, 'black': 1, 'soft': 1, 'cover': 1, '5': 1, 'x': 1, '825': 1}), Counter({'365': 1, 'cats': 1, 'color': 1, 'pageaday': 1, 'calendar': 1, '2016': 1}), Counter({'sierra': 1, 'club': 1, 'engagement': 1, 'calendar': 1, '2016': 1})]


In [20]:
# token frequencies in total vocab
tok_freq = Counter()
for title in train['titles']:
    for i in title:
        tok_freq[i] += 1
        
print(tok_freq)

Counter({'the': 92279, 'and': 54318, 'of': 50964, 'a': 31373, 'to': 27820, 'for': 21410, 'in': 21233, 'guide': 16053, 'edition': 12632, 'book': 8153, 'with': 7788, 'your': 7387, 'from': 6541, 'series': 6380, 'on': 6010, 'how': 5860, 'an': 5137, 'new': 4838, 'life': 4712, 'volume': 4065, 'world': 4045, 'history': 3970, 'you': 3810, '1': 3379, 'by': 3241, 'recipes': 3184, 'american': 3049, 'travel': 2950, '2': 2744, 'complete': 2722, 'law': 2715, 'story': 2633, 'art': 2597, 'handbook': 2544, 'health': 2525, 'my': 2460, 'books': 2402, 'science': 2250, 'calendar': 2204, 'at': 2170, 'best': 2056, 'novel': 2056, 'practice': 2001, 'i': 1946, 'guides': 1931, 'cookbook': 1907, 'map': 1902, 'that': 1863, 'vol': 1834, 'love': 1791, 'about': 1775, '2016': 1740, 'what': 1738, 'great': 1713, 'stories': 1694, 'business': 1677, 'war': 1658, 'more': 1627, 'is': 1582, 'one': 1577, 'manual': 1571, 'america': 1565, 'second': 1553, 'introduction': 1547, 'modern': 1542, 'it': 1534, 'management': 1533, 'firs

In [21]:
# group data by genre
# could probably be done MUCH nicer, but couldn't get for-loop to work...

grouped = train.groupby(train.genres)

Calendars = grouped.get_group("Calendars")
Comics = grouped.get_group("Comics & Graphic Novels")
Test = grouped.get_group("Test Preparation")
Mystery = grouped.get_group("Mystery, Thriller & Suspense")
SciFi = grouped.get_group("Science Fiction & Fantasy")
Romance = grouped.get_group("Romance")
Humor = grouped.get_group("Humor & Entertainment")
Literature = grouped.get_group("Literature & Fiction")
LGBTQ = grouped.get_group("Gay & Lesbian")
Engineering = grouped.get_group("Engineering & Transportation")
Food = grouped.get_group("Cookbooks, Food & Wine")
Crafts = grouped.get_group("Crafts, Hobbies & Home")
Arts = grouped.get_group("Arts & Photography")
Education = grouped.get_group("Education & Teaching")
Parenting = grouped.get_group("Parenting & Relationships")
SelfHelp = grouped.get_group("Self-Help")
Computers = grouped.get_group("Computers & Technology")
Medical = grouped.get_group("Medical Books")
Science = grouped.get_group("Science & Math")
Health = grouped.get_group("Health, Fitness & Dieting")
Business = grouped.get_group("Business & Money")
Law = grouped.get_group("Law")
Biographies = grouped.get_group("Biographies & Memoirs")
History = grouped.get_group("History")
Politics = grouped.get_group("Politics & Social Sciences")
Reference = grouped.get_group("Reference")
Bibles = grouped.get_group("Christian Books & Bibles")
Religion = grouped.get_group("Religion & Spirituality")
Sports = grouped.get_group("Sports & Outdoors")
Teen = grouped.get_group("Teen & Young Adult")
Childrens = grouped.get_group("Children's Books")
Travel = grouped.get_group("Travel")

GenreGroups = [Calendars['titles'], Comics['titles'], Test['titles'], Mystery['titles'], SciFi['titles'], 
               Romance['titles'], Humor['titles'], Literature['titles'], LGBTQ['titles'], Engineering['titles'], 
               Food['titles'], Crafts['titles'], Arts['titles'], Education['titles'], Parenting['titles'], 
               SelfHelp['titles'], Computers['titles'], Medical['titles'], Science['titles'], Health['titles'], 
               Business['titles'], Law['titles'], Biographies['titles'], History['titles'], Politics['titles'], 
               Reference['titles'], Bibles['titles'], Religion['titles'], Sports['titles'], Teen['titles'], 
               Childrens['titles'], Travel['titles']]

In [22]:
# token frequencies in each genre
genre_count = []
for g in GenreGroups:
    genre_title = Counter()
    for title in g:
        for i in title:
            genre_title[i] += 1
    genre_title = dict(genre_title)
    genre_count.append(genre_title)

In [23]:
# token frequencies in each genre
genre_count[0] # genres in same order as GenreGroups

{'moms': 9,
 'family': 12,
 'wall': 821,
 'calendar': 1979,
 '2016': 1120,
 'doug': 1,
 'the': 377,
 'pug': 4,
 'moleskine': 31,
 'weekly': 67,
 'notebook': 19,
 '12m': 6,
 'large': 23,
 'black': 27,
 'soft': 7,
 'cover': 31,
 '5': 16,
 'x': 28,
 '825': 15,
 '365': 73,
 'cats': 39,
 'color': 12,
 'pageaday': 55,
 'sierra': 2,
 'club': 3,
 'wilderness': 9,
 'ansel': 2,
 'adams': 2,
 'dilbert': 5,
 'daytoday': 85,
 'mary': 5,
 'engelbreit': 4,
 'deluxe': 34,
 'never': 3,
 'give': 1,
 'up': 10,
 'llewellyns': 4,
 'witches': 3,
 'datebook': 2,
 'amy': 3,
 'knapp': 2,
 'big': 11,
 'grid': 4,
 'essential': 1,
 'organization': 1,
 'and': 141,
 'communication': 1,
 'tool': 1,
 'for': 61,
 'entire': 1,
 'outlander': 2,
 'national': 19,
 'park': 2,
 'foundation': 1,
 'your': 11,
 'year': 56,
 'mindful': 2,
 'coloring': 14,
 'through': 28,
 'seasons': 5,
 'enjoy': 1,
 'joy': 1,
 'extra': 3,
 '75': 3,
 '10': 3,
 'susan': 5,
 'branch': 5,
 'dog': 25,
 'gallery': 8,
 'maxine': 6,
 'yearinabox': 6,
 

In [25]:
# number of words in a class
len(genre_count[0]) # first genre

2383

In [26]:
# number of total vocabulary (training set)
V = len(Counter(tok_freq))
print(V)

74955


In [27]:
# number of titles (in training set)
N_titles = len(train)
print(N_titles)

166341


In [28]:
# number of titles in each genre
N_genre = train['genres'].value_counts()
print(N_genre[:5])

Travel                       14769
Children's Books             10853
Medical Books                 9629
Health, Fitness & Dieting     9526
Business & Money              8024
Name: genres, dtype: int64


In [29]:
# priors of each genre (probability of title being in specific genre)
prob_title = []
for g in range(len(N_genre)):
    prob = N_genre[g] / N_titles
    prob_title.append(prob)

zipped_values = zip(df1.genre.unique(), prob_title)
prior = list(zipped_values)

print(prior[:5]) # test

[('Calendars', 0.08878749075693906), ('Comics & Graphic Novels', 0.0652454896868481), ('Test Preparation', 0.05788711141570629), ('Mystery, Thriller & Suspense', 0.05726790147949092), ('Science Fiction & Fantasy', 0.04823825755526298)]


In [30]:
# likelihoods of each word (probability of word being in specific genre)

# create empty dataframe
likelihood = pd.DataFrame(columns=df1.genre.unique(), index=dict.keys(tok_freq))

len_gc = -1
for i in genre_count: # loop through genres
    len_gc += 1 # create genre index
    for word in i: 
        p = (genre_count[len_gc][word] + 1) / (len(genre_count[len_gc]) + V) # probability
        likelihood.loc[word, likelihood.columns[len_gc]] = p # replace NaN in dataframe with p

In [55]:
# now fill likelihoods of words that haven't appeared in genre

len_gc = -1
for c, v in likelihood.iteritems(): # loop through genres in dataframe
  len_gc += 1 # create genre index
  p = 1 / (len(genre_count[len_gc]) + V) # probability of word not appearing in that genre
  likelihood[c].fillna(p, inplace=True) # replace remaining NaNs in dataframe with p

In [80]:
likelihood

,Calendars,Comics & Graphic Novels,Test Preparation,"Mystery, Thriller & Suspense",Science Fiction & Fantasy,Romance,Humor & Entertainment,Literature & Fiction,Gay & Lesbian,Engineering & Transportation,"Cookbooks, Food & Wine","Crafts, Hobbies & Home",Arts & Photography,Education & Teaching,Parenting & Relationships,Self-Help,Computers & Technology,Medical Books,Science & Math,"Health, Fitness & Dieting",Business & Money,Law,Biographies & Memoirs,History,Politics & Social Sciences,Reference,Christian Books & Bibles,Religion & Spirituality,Sports & Outdoors,Teen & Young Adult,Children's Books,Travel
moms,0.000129,0.000013,0.000013,0.000013,0.000013,0.000025,0.000060,0.000012,0.000051,0.000013,0.000132,0.000023,0.000036,0.000038,0.000329,0.000038,0.000012,0.000024,0.000024,0.000093,0.000024,0.000024,0.000024,0.000012,0.000025,0.000025,0.000096,0.000024,0.000024,0.000024,0.000058,0.000045
family,0.000168,0.000229,0.000180,0.000090,0.000050,0.000427,0.000358,0.000372,0.000412,0.000050,0.002282,0.000421,0.000253,0.000167,0.001671,0.000189,0.000086,0.000544,0.000189,0.001250,0.000711,0.001117,0.000825,0.000319,0.000212,0.000375,0.000996,0.000334,0.000097,0.000385,0.000652,0.001042
wall,0.010629,0.000013,0.000026,0.000064,0.000050,0.000013,0.000250,0.000048,0.000013,0.000038,0.000024,0.000245,0.000241,0.000026,0.000013,0.000025,0.000024,0.000047,0.000059,0.000058,0.000759,0.000073,0.000109,0.000177,0.000025,0.000188,0.000120,0.000155,0.000158,0.000072,0.000221,0.000385
calendar,0.025602,0.000013,0.000013,0.000013,0.000013,0.000025,0.000346,0.000048,0.000013,0.000163,0.000048,0.000315,0.000433,0.000013,0.000038,0.000088,0.000086,0.000012,0.000118,0.000058,0.000059,0.000012,0.000012,0.000012,0.000037,0.000050,0.000096,0.000250,0.000230,0.000024,0.000023,0.000396
2016,0.014495,0.000013,0.001093,0.000013,0.000025,0.000013,0.000381,0.000072,0.000039,0.000501,0.000048,0.000479,0.000433,0.000231,0.000089,0.000139,0.000526,0.000615,0.000071,0.000093,0.000332,0.000121,0.000024,0.000035,0.000037,0.000338,0.000168,0.000286,0.000255,0.000012,0.000082,0.001291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4829,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000013,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000023
50mile,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000013,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000023
pinpointing,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000013,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000023
historymapped,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000013,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000023


In [81]:
# test NB using test data

# predictions on word by word basis using words in titles of test data
# for each word in test title do: prior * likelihood^2

# this does prior * likelihood!! -> square likelihood

predictors = pd.DataFrame(columns=df1.genre.unique(), index=dict.keys(tok_freq)) # create empty dataframe
len_gc = -1
for i in prior:
  len_gc += 1 # create genre index
  # multiply values in each column with prior of that genre
  predictors = likelihood.loc[:, df1.genre.unique().tolist()] ** 2 # square likelihoods
  predictors = predictors.loc[:, df1.genre.unique().tolist()] * np.array(prior[len_gc][1]) # multiply by priors

In [82]:
predictors

,Calendars,Comics & Graphic Novels,Test Preparation,"Mystery, Thriller & Suspense",Science Fiction & Fantasy,Romance,Humor & Entertainment,Literature & Fiction,Gay & Lesbian,Engineering & Transportation,"Cookbooks, Food & Wine","Crafts, Hobbies & Home",Arts & Photography,Education & Teaching,Parenting & Relationships,Self-Help,Computers & Technology,Medical Books,Science & Math,"Health, Fitness & Dieting",Business & Money,Law,Biographies & Memoirs,History,Politics & Social Sciences,Reference,Christian Books & Bibles,Religion & Spirituality,Sports & Outdoors,Teen & Young Adult,Children's Books,Travel
moms,1.046322e-10,1.012374e-12,1.035316e-12,1.030036e-12,9.886194e-13,3.954179e-12,2.221890e-11,9.028382e-13,1.655995e-11,9.835938e-13,1.092592e-10,3.416442e-12,8.165509e-12,9.266999e-12,6.781408e-10,8.987037e-12,9.356757e-13,3.499424e-12,3.502902e-12,5.363808e-11,3.518618e-12,3.692059e-12,3.687310e-12,8.754149e-13,3.893948e-12,3.920895e-12,5.762788e-11,3.556465e-12,3.683643e-12,3.621698e-12,2.123254e-11,1.283553e-11
family,1.768285e-10,3.280093e-10,2.029220e-10,5.047179e-11,1.581791e-11,1.142758e-09,7.998805e-10,8.676275e-10,1.059837e-09,1.573750e-11,3.259716e-08,1.106927e-09,4.001100e-10,1.740137e-10,1.747918e-08,2.246759e-10,4.584811e-11,1.851195e-09,2.241857e-10,9.775540e-09,3.166756e-09,7.812396e-09,4.262531e-09,6.381774e-10,2.813377e-10,8.822014e-10,6.203101e-09,6.970672e-10,5.893830e-11,9.271548e-10,2.663409e-09,6.789998e-09
wall,7.069833e-07,1.012374e-12,4.141264e-12,2.575091e-11,1.581791e-11,9.885448e-13,3.919414e-10,1.444541e-11,1.034997e-12,8.852344e-12,3.611873e-12,3.766627e-10,3.629115e-10,4.118666e-12,1.003167e-12,3.994239e-12,3.742703e-12,1.399770e-11,2.189314e-11,2.095238e-11,3.603065e-09,3.322853e-11,7.466804e-11,1.969683e-10,3.893948e-12,2.205504e-10,9.004356e-11,1.502606e-10,1.556339e-10,3.259528e-11,3.065978e-10,9.273673e-10
calendar,4.102003e-06,1.012374e-12,1.035316e-12,1.030036e-12,9.886194e-13,3.954179e-12,7.474438e-10,1.444541e-11,1.034997e-12,1.662274e-10,1.444749e-11,6.226465e-10,1.175833e-09,1.029667e-12,9.028502e-12,4.892942e-11,4.584811e-11,8.748560e-13,8.757256e-11,2.095238e-11,2.199136e-11,9.230147e-13,9.218276e-13,8.754149e-13,8.761383e-12,1.568358e-11,5.762788e-11,3.921003e-10,3.324488e-10,3.621698e-12,3.397206e-12,9.827206e-10
2016,1.314852e-06,1.012374e-12,7.480159e-09,1.030036e-12,3.954478e-12,9.885448e-13,9.100862e-10,3.250217e-11,9.314969e-12,1.573750e-09,1.444749e-11,1.435760e-09,1.175833e-09,3.336120e-10,4.915518e-11,1.208257e-10,1.730064e-09,2.365611e-09,3.152612e-11,5.363808e-11,6.896491e-10,9.230147e-11,3.687310e-12,7.878734e-12,8.761383e-12,7.145832e-10,1.764854e-10,5.121310e-10,4.061217e-10,9.054246e-13,4.161577e-11,1.042566e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4829,1.046322e-12,1.012374e-12,1.035316e-12,1.030036e-12,9.886194e-13,9.885448e-13,8.887561e-13,9.028382e-13,1.034997e-12,9.835938e-13,9.029683e-13,8.541105e-13,9.072788e-13,1.029667e-12,1.003167e-12,9.985597e-13,9.356757e-13,8.748560e-13,8.757256e-13,8.380950e-13,8.796545e-13,9.230147e-13,9.218276e-13,8.754149e-13,9.734870e-13,9.802238e-13,9.004356e-13,8.891163e-13,9.209109e-13,9.054246e-13,8.493015e-13,3.208884e-12
50mile,1.046322e-12,1.012374e-12,1.035316e-12,1.030036e-12,9.886194e-13,9.885448e-13,8.887561e-13,9.028382e-13,1.034997e-12,9.835938e-13,9.029683e-13,8.541105e-13,9.072788e-13,1.029667e-12,1.003167e-12,9.985597e-13,9.356757e-13,8.748560e-13,8.757256e-13,8.380950e-13,8.796545e-13,9.230147e-13,9.218276e-13,8.754149e-13,9.734870e-13,9.802238e-13,9.004356e-13,8.891163e-13,9.209109e-13,9.054246e-13,8.493015e-13,3.208884e-12
pinpointing,1.046322e-12,1.012374e-12,1.035316e-12,1.030036e-12,9.886194e-13,9.885448e-13,8.887561e-13,9.028382e-13,1.034997e-12,9.835938e-13,9.029683e-13,8.541105e-13,9.072788e-13,1.029667e-12,1.003167e-12,9.985597e-13,9.356757e-13,8.748560e-13,8.757256e-13,8.380950e-13,8.796545e-13,9.230147e-13,9.218276

In [83]:
predictors.loc[["moms"]]

,Calendars,Comics & Graphic Novels,Test Preparation,"Mystery, Thriller & Suspense",Science Fiction & Fantasy,Romance,Humor & Entertainment,Literature & Fiction,Gay & Lesbian,Engineering & Transportation,"Cookbooks, Food & Wine","Crafts, Hobbies & Home",Arts & Photography,Education & Teaching,Parenting & Relationships,Self-Help,Computers & Technology,Medical Books,Science & Math,"Health, Fitness & Dieting",Business & Money,Law,Biographies & Memoirs,History,Politics & Social Sciences,Reference,Christian Books & Bibles,Religion & Spirituality,Sports & Outdoors,Teen & Young Adult,Children's Books,Travel
moms,1.046322e-10,1.012374e-12,1.035316e-12,1.030036e-12,9.886194e-13,3.954179e-12,2.221890e-11,9.028382e-13,1.655995e-11,9.835938e-13,1.092592e-10,3.416442e-12,8.165509e-12,9.266999e-12,6.781408e-10,8.987037e-12,9.356757e-13,3.499424e-12,3.502902e-12,5.363808e-11,3.518618e-12,3.692059e-12,3.687310e-12,8.754149e-13,3.893948e-12,3.920895e-12,5.762788e-11,3.556465e-12,3.683643e-12,3.621698e-12,2.123254e-11,1.283553e-11


In [95]:
# output genre that has highest prediction value for input word
maxValueIndexObj = predictors.idxmax(axis=1)
print(maxValueIndexObj[["rose"]])

rose    Teen & Young Adult
dtype: object


In [62]:
# function with input title from test data
# predict on word by word basis -> choose highest predictor value for each word


pandas.core.series.Series

## BERT classifier

In [ ]:
!pip install transformers

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import torch

# Preliminaries

from torchtext.data import Field, TabularDataset, BucketIterator, Iterator

# Models

import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification

# Training

import torch.optim as optim

# Evaluation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# Model parameter
MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

In [ ]:
# Fields

label_field = Field(sequential=False, use_vocab=False, batch_first=True, dtype=torch.float)
text_field = Field(use_vocab=False, tokenize=tokenizer.encode, lower=False, include_lengths=False, batch_first=True,
                   fix_length=MAX_SEQ_LEN, pad_token=PAD_INDEX, unk_token=UNK_INDEX)
fields = [('label', label_field), ('title', text_field), ('text', text_field), ('titletext', text_field)]

In [ ]:
# split data into test and train data -> use test and train variables

In [ ]:
# Iterators -> what is this?

In [ ]:
# TabularDataset

train, valid, test = TabularDataset.splits(path=source_folder, train='train.csv', validation='valid.csv',
                                           test='test.csv', format='CSV', fields=fields, skip_header=True)

NameError: ignored

In [ ]:
# Iterators

train_iter = BucketIterator(train, batch_size=16, sort_key=lambda x: len(x.text),
                            device=device, train=True, sort=True, sort_within_batch=True)
valid_iter = BucketIterator(valid, batch_size=16, sort_key=lambda x: len(x.text),
                            device=device, train=True, sort=True, sort_within_batch=True)
test_iter = Iterator(test, batch_size=16, device=device, train=False, shuffle=False, sort=False)